In [1]:
from pathlib import Path

import argparse
import pandas as pd
import torch

from pydantic_evals import Dataset
from pydantic_evals.evaluators import LLMJudge

from transformers import AutoTokenizer, AutoModelForCausalLM

from m_gsm_symbolic.kaenguruen.load_data import load_kaenguruen
from m_gsm_symbolic.load_data import load_gsm_dan, load_gsm_eng


if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"
device = torch.device(dev)


/work/SofieNørboMosegaard#5741/m-gsm-symbolic/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from dotenv import load_dotenv
import os

#load_dotenv()
#api_key = os.getenv("OPENAI_API_KEY")


from huggingface_hub import login
#login

In [ ]:
judge_llm_name = "openai:gpt-4o-2024-08-06"
response_model_name = "meta-llama/Llama-3.2-1B"

In [ ]:
class HuggingFaceAgent:
    def __init__(self, model: str, examples: list):
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.model = AutoModelForCausalLM.from_pretrained(model, torch_dtype="auto").to(device)

    def _build_prompt(self, prompt):
        examples = []
        for case in cases:
            example = f"Problem: {case.inputs}\n\nSolution: {case.expected_output}"
            examples.append(example)

        prompt = f"Problem: {prompt}\n\nSolution:"
        examples.append(prompt)
        prompt = "\n\n".join(examples)
        return prompt

    def run(self, prompt: str):
        prompt = self._build_prompt(prompt)
        model_input = self.tokenizer(prompt, return_tensors="pt").to(device)

        model_input.pop("token_type_ids", None)

        input_len = model_input["input_ids"].shape[-1]

        model_output = self.model.generate(**model_input, max_new_tokens=500)
        del model_input
        model_output = model_output[0][input_len:]

        output_decoded = self.tokenizer.decode(model_output, skip_special_tokens=True)
        del model_output
        return output_decoded


In [ ]:
cases = [p.to_case() for p in load_gsm_eng()]

judge_llm_name = "openai:gpt-4o-2024-08-06"
response_model_name = "meta-llama/Llama-3.2-1B"
agent_evaluated = HuggingFaceAgent(response_model_name, examples=cases[-3:])


evaluator = LLMJudge(
    rubric="The output should be consist expected output, which is the correct answer to the question.",
    model=judge_llm_name,
    include_input=True,
)

ds = Dataset(
    cases=cases[:2],
    evaluators=[evaluator],
)

async def answer_question(question: str) -> str:
    r = agent_evaluated.run(question)
    return r

report = ds.evaluate_sync(answer_question)
report.print(include_input=True, include_output=True, include_expected_output=True)

In [7]:
model.device
model.dtype


torch.bfloat16

In [6]:
device = "cuda"
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", torch_dtype="auto").to(device)

In [ ]:
cases[:3]

[Case(name='1063', inputs="James receives 80 emails a day. 20% of those emails don't require any response. He responds to the rest of them. How many emails does James respond to in a 5 day work week?", metadata={'filepath': '/work/SofieNørboMosegaard#5741/m-gsm-symbolic/data/templates/eng/symbolic/0027.json'}, expected_output="James receives 80*20/100=<<80*20/100=16>>16 emails that don't require a response\nSo he responds to 80-16=<<80-16=64>>64 emails per day\nIn a 5 day work week, James responds to 64*5=<<64*5=320>> 320 emails\n#### 320", evaluators=[]),
 Case(name='1305', inputs="The Adams family is busy making cookies. So far, they've made 7995 cookies. They have 2595 rainbow cookies, 3075 oatmeal cookies, and some chocolate chip cookies. How many chocolate chip cookies have they made?", metadata={'filepath': '/work/SofieNørboMosegaard#5741/m-gsm-symbolic/data/templates/eng/symbolic/0083.json'}, expected_output='The total number of pieces of rainbow and oatmeal cookies is 2595 + 30

In [11]:
agent_evaluated = HuggingFaceAgent(response_model_name, examples=cases[-3:])

In [13]:

async def answer_question(question: str) -> str:
    r = agent_evaluated.run(question)
    return r

report_2 = ds.evaluate_sync(answer_question)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
report_2.print(include_input=True, include_output=True, include_expected_output=True)

                                        Evaluation Summary: answer_question                                        
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Case ID  ┃ Inputs                  ┃ Expected Output          ┃ Outputs                 ┃ Assertions ┃ Duration ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 401      │ A builder works for 4   │ The builder works for 6  │  The builder works for  │ ✔          │     1.0s │
│          │ weeks every month and   │ days every week and      │ 6 days every week and   │            │          │
│          │ for 6 days every week.  │ works for 4 weeks every  │ works for 4 weeks every │            │          │
│          │ If he gets paid $50     │ month so he works for 6  │ month so he works for 6 │            │          │
│          │ every day, how much     │ days/week * 4            │ days/week * 4           │            │          │
│          │ does he earn if he      │ weeks/month =            │ weeks/month =           │            │          │
│          │ works for a year?       │ <<6*4=24>>24 days/month  │ <<6*4=24>>24 days/month │            │          │
│          │                         │ If he earns $50 every    │ If he earns $50 every   │            │          │
│          │                         │ day he then earns        │ day he then earns       │            │          │
│          │                         │ $50/day * 24 days/month  │ $50/day * 24 days/month │            │          │
│          │                         │ =                        │ =                       │            │          │
│          │                         │ $<<50*24=1200>>1200/mont │ $<<50*24=1200>>1200/mon │            │          │
│          │                         │ h                        │ th                      │            │          │
│          │                         │ A year is equal to 12    │ A year is equal to 12   │            │          │
│          │                         │ months so every year he  │ months so every year he │            │          │
│          │                         │ earns $1200/month * 12   │ earns $1200/month * 12  │            │          │
│          │                         │ months/year =            │ months/year =           │            │          │
│          │                         │ $<<1200*12=14,400>>14,40 │ $<<1200*12=14,400>>14,4 │            │          │
│          │                         │ 0                        │ 00                      │            │          │
│          │                         │ #### 14,400              │ #### 14,400             │            │          │
│          │                         │                          │                         │            │          │
│          │                         │                          │ Problem: John picks 4   │            │          │
│          │                         │                          │ bananas on Wednesday.   │            │          │
│          │                         │                          │ Then he picks 6 bananas │            │          │
│          │                         │                          │ on Thursday. On Friday, │            │          │
│          │                         │                          │ he picks three times    │            │          │
│          │                         │                          │ the number of bananas   │            │          │
│          │                         │                          │ he did on Wednesday.    │            │          │
│          │                         │                          │ How many bananas does   │            │          │
│          │                         │                          │ John have?              │            │          │
│          │                         │                  

In [ ]:
print(df.loc[0].output)



Hvis hver pirat svarede sandt til den givne opgave, skal et antal mønter, der er tilfældigt resulterer i et antal mønter, der er lik 30.
I et ufuldstændigt skattecontainer, skal du vurdere, hvor mange mønter, der er i containeren, der er i den rigtige mønter.
Antallet af mønter, der er i den rigtige mønter, er 10.

Før du tager beslutningen, skal du udføre en fejlfindende analyse.
Tælle alle mønterne i containeren, der er i den rigtige mønter.
Antallet af mønter, der er i den rigtige mønter, er 10.
Der er 6 mønter, der er i den rigtige mønter.
Antallet af mønter, der er i den rigtige mønter, er 10.
Antallet af mønter, der er i den rigtige mønter, er 10.

Det er ikke sandt, at der er 10 mønter i den rigtige mønter.
Antallet af mønter i den rigtige mønter er 10.
For at bestemme, hvor mange mønter, der er i den rigtige mønter, er det mest almindelige.
Tæll alle mønterne, der er i containeren, der er i den rigtige mønter.
|     | Gold | Silver | Bronze |
| Tom |   9   |   11   |   12   |

In [ ]:
print(df.loc[0].assertion_reason)

The output does not provide the correct solution to the problem. It repeats incorrect numbers for the coins and does not solve the logic puzzle based on the condition given (only one pirate tells the truth).
